In [1]:
from openpyxl import load_workbook
from google.cloud import storage
from pyspark.sql import *
from pyspark.sql.functions import *
import requests
import pandas as pd
from io import BytesIO
import hashlib
from pyspark.sql.types import *
from pyspark.sql import Window as W
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)  

In [2]:
from pyspark.sql.utils import *


In [3]:
# Create Spark session
spark = SparkSession.builder \
    .appName("DateRangeExample") \
    .getOrCreate()

In [4]:
def get_max_date(trgt_path_processed):
    max_date="0"
    try:
        # Try to read the Parquet file
        df = spark.read.parquet(trgt_path_processed)
        # Check if DataFrame is empty
        if df.head(1):
            df.createOrReplaceTempView("vw_max")
            max_date_s = spark.sql("select max(datesk) as max_date from vw_max").collect()[0][0]
            max_date = max_date_s[0:4]+"-"+max_date_s[4:6]+"-"+max_date_s[6:8]
            return max_date
        else:
            max_date = "0"
            return max_date
    except AnalysisException:
        return max_date